# 3a Corridor Analysis

This module calculates corridor-level metrics (vmt, pmt, delay, ghg, and costs). The resulting Excel files contain a "Summary" sheet with banner metrics and a sheet with delay by milepost. Additional sheets can be added that can be used to verify calculations. The corridor objects produced by this code will also be written to binary files (with .dat extension) and will be used in the commute analysis. The module also produces the delay heatmaps and stampgraphs. To run this module, edit the inputs in the *0_Interface.xlsx* file and then step through the Main Code block.

Before running this module, there are a few files that need to be prepared. First, the Speed and Volume per Hour contour files for each corridor need to be placed in an appropriate folder; the default location is *./[ccr]/1_Data/[region]/1_Corridor Data/[year]*, but this can be changed in the *Filepaths* sheet of the *0_Interface.xlsx* file, if desired.

The corridor config files need to be prepared (see *2_Truck Percentages.ipynb*) and placed in an appropriate folder (default location is *./[ccr]/0_Inputs/[region]* but this may also be changed if desired). Finally, the *Statewide_config.xlsx* file (in *./[ccr]/0_Inputs/* by default) needs to be prepared by adding cost and ghg information to the 'Cost' and 'GHG' sheets, respectively.

You can choose which corridors to **exclude** during batch processing in the *Batch* sheet of the *0_Interface.xlsx* file. If left blank, all corridors of all regions will be processed during batch processing.

## Inputs

The inputs for this module are:

Base Year, Current Year : the base and current analysis years for the current analysis. (e.g. 2015, 2017)

CCR : the current CCR and name of the main folder for the current CCR (e.g. 'CCR 18')

Analyst : analyst's name

Suffix : this is a tag to add to the filenames of the Excel and binary outputs of this module. For example, if Suffix = '_8-20-2018' then the Excel outputfile for I-5 in NWR would be named *5 NWR_8-20-2018.xlsx*.  
*Note - The *4b_Commute_Analysis* module and the SummaryStat spreadsheets use the outputs from this step as inputs. While using a suffix is useful to clearly distinguish files when comparing different inputs/assumptions, the final versions (those used in subsequent steps) **must** be saved as [corridor] [region].[extension]. with no suffix* 

Excel Variables : these are variables to write to the Excel spreadsheet so that intermediate calculations can be checked. The variable names can be any of the contour-file variables or variables calculated from the contour files. Variables include speed, volume, delay, cd (cost of delay), ghg, vmt, pmt, cong (congestion), cc (cost of congestion), etc. The resulting sheets will include tables of the calculated variable in a contour file format (i.e. time as rows and milepost as columns).

Heatmap Variables, Lineplot Variables : these are the variables to plot in heatmaps and lineplots for each corridor. Only delay is published, but it is also useful to output volume and speed for deeper analysis and understanding what is happening.

Heatmap Norm : this is the maximum delay (in veh-hrs per 5-min period per half-mile road section), used to normalize the heatmaps. For example, if the maximum in all regions is 11.4, then I would use this value for all corridors to be able to compare delay intensity across corridors. Black in each heatmap would then correspond to 11.4 veh-hrs/5-min/half-mile, white would correspond to 0, and the shade of gray in between would represent a number between 0 and 11.4 (with lighter being closer to 0 and darker being closer to 11.4).

Update Plotting : this is a boolean argument indicating whether to re-load the plot parameters (plot_params attribute of the Corridor objects, read from the *Plotting* sheet of the *[corridor]_[region]_config.xlsx* file.

## Main Code Block

#### Import Dependencies

In [ ]:
#---------------------------- import dependencies -----------------------------
import os
import pickle
import time
from pymas.corridor_tools import *
from pymas.pymas_classes import Corridor, LoopGroup

# disable warnings
import warnings
warnings.filterwarnings('ignore')

#### Create Corridor object(s) and write to Excel and .dat

This cell creates Corridor objects for all of the corridors indicated in the *Batch* sheet of *0_Interface.xlsx*. Building an object entails reading in the relevant contour files (speed & volume) and configuration files (statewide and corridor-specific) and then calculating several metrics using these files. It then stores these objects in a dictionary called "objs" to hold all of the data.

This cell will also report the maximum delay out of the NWR, OR, and SWR corridor objects that are included in the batch; this maximum will be used when plotting the heatmaps to normalize the scale. This maximum is then printed at the end of the output and this number can be entered as *Heatmap Norm* if appropriate.

Finally, this cell writes the objects to Excel (including the Summary and Delay_Comparison sheets plus any variables indicated with the *Excel Variables* input. It will also write the Corridor objects to binary files to be used in *3b_Corridor_Analysis_loops* and *4b_Commute_Analysis* modules.

**Note for future users:**  
*If ER data is added to Tracflow and we get contour data on a 0.5 mile basis, change the line below from "if reg in ['NWR', 'OR', 'SWR']:" to "if reg in ['NWR', 'OR', 'SWR', 'ER']:"*

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('3a', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']
suffix = inputs['suffix']
xl_vars = inputs['xl_vars']

reg_cors = get_batchlist()

# define empty list and dict to store results
max_delay = []
objs = {}

for reg, cor_list in reg_cors.iteritems():
    if len(cor_list) > 0:
        print('\nProcessing %s'%reg)
        paths = define_paths(ccr, reg, base_year, curr_year)
    
    for cor in cor_list:
        name = '%s %s'%(cor, reg)
        print('Building %s Corridor object'%name)
        obj = Corridor(cor, reg, paths, base_year, curr_year, analyst)
        
        print('Writing %s to file'%name)
        obj.export_excel(xl_vars, suffix)
        obj.export_dat(suffix)        
        
        objs['%s %s'%(cor,reg)] = obj
        
        if reg in ['NWR', 'OR', 'SWR']:
            max_delay.append(obj.max_delay)

try:    
    print('\nThe maximum delay is %s'%(max(max_delay)))
except ValueError:
    print('\nThe maximum delay is N/A')

#### Build objs dictionary from .dat files

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('3a', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
suffix = inputs['suffix']

reg_cors = get_batchlist()

try:
    objs
except NameError:
    objs = {}

for reg, cor_list in reg_cors.iteritems():
    if len(cor_list) > 0:
        print('\nLoading %s'%reg)  
        paths = define_paths(ccr, reg, base_year, curr_year)
    
    for cor in cor_list:
        name = '%s %s'%(cor, reg)

        if name not in objs.keys():
            try:
                objs[name] = read_object(name, 'Corridor', paths, suffix)
            except:
                print('Could not find %s%s.dat Corridor object'%(name, suffix))
                continue

print('\nDone building objs dictionary.')

#### Plot heatmaps and line graphs

This section exports the heatmaps and linegraphs for the corridor. Even though only delay plots are published, it is useful to plot volume and speed also to help identify data issues and to help explain what is really happening.

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('3a', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']
suffix = inputs['suffix']
hm_vars = inputs['hm_vars']
line_vars = inputs['line_vars']
update = inputs['update_pp']
try:
    norm = inputs['hm_norm'] if inputs['hm_norm'] else max(max_delay)
except NameError:
    if inputs['hm_norm']:
        norm = inputs['hm_norm']
    else:
        raise Exception('norm must be defined if max_delay is not defined')
    
reg_cors = get_batchlist()

try:
    objs
except NameError:
    objs = {}

for reg, cor_list in reg_cors.iteritems():
    if len(cor_list) > 0:    
        print('\nProcessing %s'%reg)  
        paths = define_paths(ccr, reg, base_year, curr_year)
    
    for cor in cor_list:
        name = '%s %s'%(cor, reg)

        if name not in objs.keys():
            try:
                objs[name] = read_object(name, 'Corridor', paths, suffix)
            except:
                print('Could not find %s%s.dat Corridor object'%(name, suffix))
                continue
        
        if update:
            objs[name].update_plot_params()
        
        print('Plotting %s'%name)
        objs[name].plot_heatmaps(hm_vars, norm)
#         objs[name].plot_heatmap(hm_vars, norm)
        objs[name].plot_line(line_vars)
    
print('\nDone plotting.')

#### Process historical data

This cell adds the daily delay and vmt for each corridor to the *[region] Historical.xlsx* file and then plots the last 10 years (if available) of historical data.

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('3a', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']
suffix = inputs['suffix']
    
reg_cors = get_batchlist()

try:
    objs
except NameError:
    objs = {}

for reg, cor_list in reg_cors.iteritems():
    if len(cor_list) == 0:    
        continue
   
    print('\nProcessing %s'%reg)  
    paths = define_paths(ccr, reg, base_year, curr_year)    
    
    for cor in cor_list:
        name = '%s %s'%(cor, reg)

        if name not in objs.keys():
            try:
                objs[name] = read_object(name, 'Corridor', paths, suffix)
            except:
                print('Could not find %s%s.dat Corridor object'%(name, suffix))
                continue
    
    write_hist(reg, objs, paths, ['delay', 'vmt'])
    plot_hist(reg, ['delay', 'vmt'], paths)
    
print('\n\nDone processing historical data')